# Sequence to sequence model with attention

#### 데이터를 [여기서](https://download.pytorch.org/tutorial/data.zip) 먼저 다운받고 현재 디렉토리에 압축을 풉니다.

In [9]:
# 필요한 모듈들을 import 해줍니다.

import unicodedata
import re, random
from io import open
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

use_cuda = torch.cuda.is_available()

### pre-processing

In [10]:
SOS_token = 1
EOS_token = 2


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "PAD", 1: "SOS", 2:"EOS"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


In [11]:
# data를 읽어오는 함수입니다. 본 예제에서는 english-to-french 데이터를 사용합니다.
# 사실 pre-processing 함수는 본인이 데이터에 맞게 직접 만들어야 하기 때문에 자세히 다루지 않겠습니다.

# data가 unicode로 되어있으므로 ascii 형식으로 바꿔주는 함수입니다.
# http://stackoverflow.com/a/518232/2809427 참조

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# letter가 아닌 character를 지우고, 대문자를 소문자로 바꾸는 함수입니다.

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


In [12]:

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # line별 데이터를 읽어옵니다.
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # pair로 나누고, normalize 해줍니다.
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # 위에서 선언한 Lang class의 instance를 만들어줍니다. 이제 각 언어별 dictionary가 생성되었습니다.
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [13]:
# torch tutorial에서는 max length=10으로 정해주고 데이터를 넣어주었지만,
# 본 예제에서는 mini-batch별로 max length를 처리해주는 후처리 과정을 거칩니다.
# prefix만 filtering 해줍니다.

def filterPair(p):
    eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
    )
    return p[0].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [14]:
# 데이터를 불러옵니다. (lang instance 불러오기)
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)

    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [15]:
# mini-batch별로 max_length 만큼의 ndarray data를 만들어 list에 추가해주는 작업입니다.
# [batch_size, max_length_per_minibatch] * num_batch

def get_data(batch_size=32):

    input_lang, output_lang, pairs = prepareData('eng', 'fra', False)
    num_batch = len(pairs) // batch_size
    en_data = []
    fr_data = []
    for i in xrange(num_batch):
        en = [pair[0] for pair in pairs[i * batch_size:(i + 1) * batch_size]]
        fr = [pair[1] for pair in pairs[i * batch_size:(i + 1) * batch_size]]
        en_batch = np.zeros([batch_size, max(map(len, en))], dtype=np.int64)
        fr_batch = np.zeros([batch_size, max(map(len, en))], dtype=np.int64)
        for j in xrange(batch_size):
            en_indexes = indexesFromSentence(input_lang, en[j])
            fr_indexes = indexesFromSentence(output_lang, fr[j])
            en_batch[j,:len(en_indexes)] = en_indexes
            fr_batch[j,:len(fr_indexes)] = fr_indexes

        en_data.append(en_batch)
        fr_data.append(fr_batch)

    return en_data, fr_data, input_lang, output_lang

## Build Graph

In [16]:
# Encoder network를 만드는 과정입니다.
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, batch_size=16):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.embedding = nn.Embedding(input_size, hidden_size)

        # nn.GRU는 [embedding_size, hidden_size, n_layers]를 보통 파라미터로 받습니다.
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=n_layers)

    def forward(self, input_, hidden):
        # gru cell에 넣어줄 embedding은 time-major입니다. [time_step=1(1로 하는게 무난합니다.), batch_size, embedding_size]

        embedded = self.embedding(input_).view(1, self.batch_size, -1)
        output = embedded

        output, hidden_state = self.gru(output, hidden)

        return output, hidden_state

    def initHidden(self):
        # [n_layers, batch_size, hidden_size]짜리 init_hidden_state를 생성하는 과정입니다.
        result = Variable(torch.zeros(self.n_layers, self.batch_size, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [21]:
# 예시입니다. [3,1]짜리 dimension을 가진 mini-batch를 받아서 [1, 3, 10]의 embedding을 만들고, [1,3,20] 짜리 output을 뱉어냅니다.
# state는 [n_layer, batch_size, hidden_size]의 사이즈를 가집니다.
e = EncoderRNN(10,20,2, batch_size=3)

o, h = e(Variable(torch.LongTensor([[1],[2],[3]])),e.initHidden())

print(o)
print(h)

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
  -0.1688  0.1311 -0.0180  0.0710  0.0374  0.1203  0.0170  0.0538  0.0322
 -0.1071  0.1147 -0.0236  0.0879  0.0522  0.0212  0.0680  0.0721  0.0510
 -0.1027  0.1665  0.0867  0.0572  0.1089 -0.0413  0.1171  0.0654  0.0649

Columns 9 to 17 
  -0.0958  0.0116 -0.2165 -0.1308 -0.0401 -0.0148  0.0158  0.0409  0.0816
 -0.0628  0.0869 -0.1104 -0.0325 -0.1468  0.0622 -0.1050  0.0589  0.1541
  0.0276  0.0840 -0.1522 -0.0176 -0.1607  0.1158 -0.1299  0.1007  0.1874

Columns 18 to 19 
   0.0451 -0.0300
  0.0916 -0.0755
  0.0586 -0.0520
[torch.FloatTensor of size 1x3x20]

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.0008  0.2141  0.2810  0.5281 -0.3966 -0.1633  0.2140 -0.3774 -0.2202
 -0.1417  0.1653  0.1261  0.1268  0.2016 -0.1923  0.1223  0.0367  0.1307
  0.0103 -0.0288  0.1859 -0.3823  0.3508 -0.1807  0.3309 -0.2962 -0.0828

Columns 9 to 17 
   0.0911  0.2426 -0.2827 -0.0962  0.0250  0.0046  0.0732  0.0970 -0.2715
  0.1024  0.0641 -0.019

In [22]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=1):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        # hidden_size * 2를 해주는 이유는 attention vector를 input과 concat시켜 그다음 input에 넣어줄 것이기 때문입니다.
        # 그림 참조
        self.gru = nn.GRU(hidden_size * 2, hidden_size)

        # Attention Mechanism
        # Attention matrix를 만드는데 필요한 layer들을 선언해줍니다.

        # linear layer를 거쳐 memory를 만들어줍니다. hs_bar입니다.
        self.get_keys = nn.Linear(hidden_size, hidden_size)

        # score를 계산할 때의 W1, W2 matrix입니다.
        self.attn = nn.Linear(hidden_size * 2, hidden_size)

        # score를 계산할 때 사용되는 learnable parameter입니다.
        self.v = Variable(torch.randn(hidden_size, 1), requires_grad=True)

        if use_cuda:
            self.v = self.v.cuda()

        # 최종적으로 Attention vector를 만들 때 사용하는 matrix입니다.
        self.get_attn_vector = nn.Linear(hidden_size * 2, hidden_size)

        # 마지막 fc layer로 prediction을 뱉습니다.
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_, hidden, enc_outs):

        seq_len, batch_size = enc_outs.data.size()[:2]

        # 전 timestep의 cell에서 뱉은 h_t를 받아옵니다.
        h_t = hidden  # hidden state of RNN

        attn_weights = []
        # Attention Mechanism
        # encoder의 각 timestep 0~s까지의 alpha_ts 값을 구하는 과정입니다.
        for j in range(seq_len):
            # memory를 만들어줍니다.
            memory = self.get_keys(enc_outs[j]) # [batch_size, hidden_size]

            # memory와 query를 concat하여 score를 계산해줍니다.
            score = F.tanh(self.attn(torch.cat([memory, h_t[0]], -1)))  # [batch_size, hidden_size]

            # v를 matrix_mul 해줍니다.
            score = torch.mm(score, self.v)  # [batch_size, hidden_size] * [hidden_size, 1(timestep)] = [batch_size, 1(timestep)]

            # softmax 값을 구하기 위해서는 encoder timestep 전부의 alpha값이 필요하므로 list에 저장해두고 for loop를 돌립니다.
            attn_weights.append(score)

        # 만들어준 list를 하나로 합칩니다.
        attn_weights = torch.cat(attn_weights, 1)  # [batch_size, max_length_encoder_minibatch]

        # alpha_ts가 vector의 형태로 나옵니다. 
        attn_weights = F.softmax(attn_weights)  # [batch_size, max_length_encoder_minibatch]

        # context vector를 만듭니다. batch별 matrix_mul입니다. 
        context_vector = torch.bmm(attn_weights.view(batch_size, 1, -1), enc_outs.view(batch_size,seq_len,-1))  # [batch_size,1,max_length_encoder_minibatch] * [batch_size,max_length_encoder_minibatch,hidden_size]

        # attention vector를 만듭니다.
        attention_vector = F.tanh(self.get_attn_vector(torch.cat([context_vector, h_t.view(batch_size,1,-1)], -1).view(batch_size, -1)))

        # decoder input embedding을 만듭니다.
        embedded = self.embedding(input_)  # [batch_size, 1, hidden_size] 

        # gru cell에 넣어줄 input을 attention vector와 concat 해줍니다.
        comb = torch.cat((attention_vector.view(batch_size, 1, -1), embedded), -1)  # [batch_size, 1, hidden_size * 2] 

        # gru cell에 input, hidden을 넣어줍니다. time_major로 바꿔줍니다. (grucell batch_first=True error)
        output, hidden = self.gru(comb.view(1, batch_size, -1), h_t.view(1, batch_size, -1))  # [1,batch_size, hidden_size*2], [1, batch_size, hidden_size]

        # 각 timestep별 최종 logit을 계산합니다.
        output = self.fc(output.view(batch_size, -1)).view(batch_size, 1, -1)  # [batch_size, 1, voca_size]

        # NLL loss logit
        output = F.log_softmax(output)

        return output, hidden, attn_weights


In [26]:
# 예시입니다. encoder length = 5, batch_size = 3
de = AttnDecoderRNN(30,10)
de(Variable(torch.LongTensor([[1],[2],[3]])), Variable(torch.zeros([1,3,30])), Variable(torch.rand([5,3,30])))

(Variable containing:
 (0 ,.,.) = 
 
 Columns 0 to 8 
   -1.0728 -1.0688 -1.1687 -1.2829 -1.2584 -0.9902 -0.9625 -0.9818 -0.9779
 
 Columns 9 to 9 
   -0.9761
 
 (1 ,.,.) = 
 
 Columns 0 to 8 
   -0.9331 -1.1643 -1.0486 -0.8193 -1.0518 -1.2424 -1.2937 -1.2349 -1.0908
 
 Columns 9 to 9 
   -1.0597
 
 (2 ,.,.) = 
 
 Columns 0 to 8 
   -1.3294 -1.0659 -1.0824 -1.2658 -1.0035 -1.0793 -1.0677 -1.0952 -1.2450
 
 Columns 9 to 9 
   -1.2849
 [torch.FloatTensor of size 3x1x10], Variable containing:
 (0 ,.,.) = 
 
 Columns 0 to 8 
    0.2160  0.0083 -0.2469 -0.1119 -0.0117 -0.3233  0.0327 -0.3268  0.1438
  -0.0880  0.3351 -0.0123 -0.1893 -0.4085 -0.2786 -0.0941  0.1479  0.2131
   0.0193  0.2138  0.1761 -0.2906 -0.3902 -0.5038 -0.0908 -0.0331  0.0792
 
 Columns 9 to 17 
   -0.0759 -0.1373  0.1704  0.2166  0.0770 -0.0065 -0.1758  0.0538 -0.3487
  -0.4121  0.1189  0.3891 -0.3790  0.0467 -0.3752  0.1173  0.2407 -0.2611
   0.2380 -0.1599  0.2320  0.0388  0.1101  0.1141  0.1856 -0.3472 -0.3219
 
 Colu

In [27]:
# sentenced의 word들을 index로 만들어주는 함수입니다.
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

# numpy data를 torch Variable로 만들어주는 함수입니다.
def np_to_variable(var):
    return Variable(torch.from_numpy(np.expand_dims(var, axis=1))).cuda() if use_cuda else Variable(torch.from_numpy(np.expand_dims(var, axis=1)))

## Training

In [30]:
# hyperparameter들을 선언해줍니다. 따로 parser를 두어도 좋습니다.
batch_size = 16
learning_rate = 0.001
num_epochs = 100
hidden_size = 128

# mini_batch 별로 split한 data를 불러옵니다.
en_data, fr_data, input_lang, output_lang = get_data(batch_size=batch_size)

# encoder instance를 만들어줍니다. GPU를 쓸때는 network도 .cuda()를 해주어야 합니다.
encoder = EncoderRNN(input_lang.n_words,hidden_size,batch_size=batch_size)
encoder = encoder.cuda() if use_cuda else encoder

# decoder instance를 만들어줍니다. 마찬가지로 GPU를 쓸때는 .cuda()를 해주어야 합니다.
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words)
decoder = decoder.cuda() if use_cuda else decoder

# optimizer들을 선언해줍니다.
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

# loss criterion을 선언해줍니다.
criterion = nn.NLLLoss()

# training을 시작합니다.
for _ in range(num_epochs):
    
    # 1 epoch 만큼입니다. 여기서 data shuffling을 해주어야 합니다. code 업데이트 추후 예정
    for j in range(len(en_data)):

        # mini_batch 마다 encoder_hidden, gradients, loss를 초기화 해줍니다.
        h = encoder.initHidden()
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        loss = 0.0

        outputs = []
        for i in range(en_data[j].shape[1]):
            # batch별 max_length만큼 encoder에 넣어줍니다.
            encoder_input = np_to_variable(en_data[j][:,i])
            encoder_input = encoder_input.cuda() if use_cuda else encoder_input
            o, h = encoder(encoder_input,h)
            outputs.append(o)
        
        # mini_batch max_length만큼의 outputs를 concat하여 하나의 tensor로 만듭니다.
        outputs = torch.cat(outputs)
        outputs = outputs.cuda() if use_cuda else outputs

        # teacher forcing은 하지 않았습니다. SOS token만 넣어주고 학습시켜줍니다.
        decoder_input = Variable(torch.LongTensor([[SOS_token] for _ in range(batch_size)]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input
        
        for k in range(fr_data[j].shape[1]):
            ou, h, at = decoder(decoder_input, h, outputs)
            topv, topi =  ou.data.topk(1)
            next_input = topi[:,0]
            
            # next_input에 이전 step의 output을 넣어줍니다.
            decoder_input = Variable(next_input)
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            
            # loss를 계산해줍니다.
            loss += criterion(ou.view(batch_size,-1), np_to_variable(fr_data[j][:, k]).contiguous().view(batch_size))
        
        # 1 mini_batch의 loss로 gradient를 계산합니다.
        loss.backward()

        # weight를 update 해줍니다.
        encoder_optimizer.step()
        decoder_optimizer.step()
        
        # logging
        if j % 100 ==0: print loss.data[0] / fr_data[j].shape[1]

Reading lines...
Read 135842 sentence pairs
Trimmed to 135842 sentence pairs
Counting words...
Counted words:
('eng', 3553)
('fra', 5393)
2.78182601929


KeyboardInterrupt: 

In [29]:
# Evaluation code는 추후 update 하겠습니다.